<a href="https://colab.research.google.com/github/TacetIrae/Final_GB/blob/main/Work_with_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Создание файлов с названиями животных
with open("House_animals.txt", "w") as f:
    f.write("Dogs\nCats\nHamster")

with open("Non_house_animals.txt", "w") as f:
    f.write("Horse\nCamels\nDonkey")

In [118]:
import sqlite3

# Создаем соединение с базой данных
conn = sqlite3.connect('Друзья человека.db')

c = conn.cursor()

# Создаем таблицу "Виды животных"
c.execute("""
CREATE TABLE IF NOT EXISTS Виды_животных (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    вид TEXT NOT NULL,
    тип TEXT NOT NULL
);
""")

# Создаем таблицу "Животные"
c.execute("""
CREATE TABLE IF NOT EXISTS Животные (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    вид TEXT NOT NULL,
    кличка TEXT NOT NULL,
    дата_рождения DATE NOT NULL,
    команда TEXT,
    возраст INTEGER
);
""")

# Заполняем таблицу "Виды животных"
c.executemany("""
INSERT INTO Виды_животных (вид, тип)
VALUES (?, ?)
""", [
    ("Собака", "Домашнее"),
    ("Кошка", "Домашнее"),
    ("Хомяк", "Домашнее"),
    ("Лошадь", "Вьючное"),
    ("Верблюд", "Вьючное"),
    ("Осел", "Вьючное"),
])

# Заполняем таблицу "Животные"
c.executemany("""
INSERT INTO Животные (вид, кличка, дата_рождения, команда)
VALUES (?, ?, ?, ?)
""", [
    ("Собака", "Бобик", "2022-01-01", "Сидеть"),
    ("Собака", "Мурка", "2021-05-05", "Лежать"),
    ("Кошка", "Хомка", "2023-03-03", "Крутиться"),
    ("Лошадь", "Гнедой", "2020-10-10", "Иго-го"),
    ("Верблюд", "Вьюрок", "2019-12-12", "Стоять"),
    ("Осел", "Серый", "2022-02-02", "Шагом"),
])

conn.commit()
conn.close()



In [120]:
conn = sqlite3.connect('Друзья человека.db')
c = conn.cursor()

# Удаляем верблюдов из таблицы "Животные"
c.execute("""
DELETE FROM Животные
WHERE вид = (SELECT вид FROM Виды_животных WHERE вид = 'Верблюд');
""")

conn.commit()
conn.close()




In [119]:
conn = sqlite3.connect('Друзья человека.db')
c = conn.cursor()

# Создаем "Молодые животные"
c.execute("""
CREATE TABLE IF NOT EXISTS Молодые_животные (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    животное_id INTEGER NOT NULL,
    возраст INTEGER NOT NULL
);
""")

# Переносим молодых животных в новую таблицу
c.execute("""
INSERT INTO Молодые_животные (животное_id, возраст)
SELECT id, (julianday('now') - julianday(дата_рождения))/365.25
FROM Животные
WHERE (julianday('now') - julianday(дата_рождения))/365.25 > 1
AND (julianday('now') - julianday(дата_рождения))/365.25 < 3;
""")


# Обновляем возраст животных
c.execute("""
UPDATE Животные
SET возраст = (julianday('now') - julianday(дата_рождения))/365.25;
""")

conn.commit()
conn.close()




А здесь вообще понять не могу, что не так. Сколько не исправляю, сколько не меняю, все одна ошибка. Может тенельное зрение, но хоть убейте, не могу найти проблемы.

In [123]:
conn = sqlite3.connect('Друзья человека.db')
c = conn.cursor()

# Создаем "Объединенная_таблица"
c.execute("""
CREATE TABLE IF NOT EXISTS Объединенная_таблица (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    кличка TEXT NOT NULL,
    вид INTEGER NOT NULL,
    дата_рождения DATE NOT NULL,
    команда TEXT,
    возраст INTEGER,
    предыдущая_таблица TEXT
);
""")

# Объединяем таблицы
c.execute("""
INSERT INTO Объединенная_таблица (кличка, вид, дата_рождения, команда, предыдущая_таблица)
SELECT кличка, вид, дата_рождения, команда, 'Животные'
FROM Животные;""")


c.execute("""
INSERT INTO Объединенная_таблица ( возраст, предыдущая_таблица)
SELECT возраст, 'Молодые_животные'
FROM Молодые_животные;
""")

# Удаляем старые таблицы
c.execute("""
DROP TABLE Животные;
""")
c.execute("""
DROP TABLE Молодые_животные;
""")

conn.commit()
conn.close()



IntegrityError: NOT NULL constraint failed: Объединенная_таблица.кличка

Дальше уже не сильно и рабочий код

In [ ]:
class Animal:
    def __init__(self, кличка, вид, тип, дата_рождения, команда):
        self.name = кличка
        self.species = вид
        self.type = тип
        self.birthday = дата_рождения
        self.command = команда

    def get_age(self):
        return (datetime.now() - self.birthday).days / 365.25

    def learn_command(self, new_command):
        self.command = new_command

class DomesticAnimal(Animal):
    def __init__(self, name, species, birthday, command):
        super().__init__(name, species, "Домашнее", birthday, command)

class PackAnimal(Animal):
    def __init__(self, name, species, birthday, command):
        super().__init__(name, species, "Вьючное", birthday, command)


Не поличается совместить с прошлой БД. Пока не сильно понимаю что именно сделать, чтобы все заработало.

In [ ]:
class AnimalRegistry:
    def __init__(self):
        self.animals = []

    def add_animal(self, animal):
        self.animals.append(animal)

    def get_animals_by_type(self, animal_type):
        return [animal for animal in self.animals if animal.type == animal_type]

    def get_animal_by_name(self, name):
        return next((animal for animal in self.animals if animal.name == name), None)

    def print_all_animals(self):
        for animal in self.animals:
            print(f"{animal.name} ({animal.species}) - {animal.command}")

    def print_menu():
        while True:
            print("-" * 20)
            print("Реестр домашних животных")
            print("-" * 20)
            print("1. Добавить новое животное")
            print("2. Показать всех животных")
            print("3. Найти животное по имени")
            print("4. Обучить животное команде")
            print("0. Выход")
            print("-" * 20)

            choice = input("Введите номер пункта меню: ")

            if choice == "1":
                self.add_animal()
            elif choice == "2":
                self.print_all_animals()
            elif choice == "3":
                name = input("Введите имя животного: ")
                animal = self.get_animal_by_name(name)
                if animal:
                    print(f"{animal.name} ({animal.species}) - {animal.command}")
                else:
                    print(f"Животное с именем '{name}' не найдено.")
            elif choice == "4":
                name = input("Введите имя животного: ")
                new_command = input("Введите новую команду: ")
                animal = self.get_animal_by_name(name)
                if animal:
                    animal.learn_command(new_command)
                    print(f"Животное '{name}' теперь умеет выполнять {new_command}")
                else:
                    print(f"Животное с именем '{name}' не найдено.")






In [ ]:
class AnimalCounter:
    def __init__(self):
        self.count = 0

    def __enter__(self):
        print("Заводим новое животное...")
        self.count += 1
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type is not None:
            print(f"Ошибка при заведении животного: {exc_value}")
        else:
            print(f"Новое животное успешно зарегистрировано! (Всего: {self.count})")


registry = AnimalRegistry()

with AnimalCounter() as counter:
    # Добавить новое животное
    registry.add_animal(DomesticAnimal("Барсик", "Кошка", "2023-07-07", "Лежать"))

# Вывести список всех животных
registry.print_all_animals()


Заводим новое животное...
Новое животное успешно зарегистрировано! (Всего: 1)
Барсик (Кошка) - Лежать


--------------------
Реестр домашних животных
--------------------
1. Добавить новое животное
2. Показать всех животных
3. Найти животное по имени
4. Обучить животное команде
0. Выход
--------------------
Введите номер пункта меню: 1


NameError: name 'self' is not defined